In [9]:
import os
import numpy as np
import cantera as ct
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
dfs = []

for i in range(1, 9):

    base_path = 'flame_speed_ch4_20210531_102003'
    file_path = f'/RUN_{i:06}/CKSoln_Flame_SpeedC1.ckcsv'
    path = base_path + file_path
    
    df_read = pd.read_csv(path, skiprows=1, header=None)

    columns_long = df_read.iloc[:, 0] + df_read.iloc[:, 1] 

    columns = []
    for name in columns_long:
        columns.append(name.split('_Run')[0])

    df = pd.DataFrame(data=df_read.iloc[:, 2:].T.to_numpy(), columns=columns)
    
    dfs.append(df)

In [4]:
print(columns)

['Distance', 'Pressure', 'Mass_density', 'Axial_velocity', 'Cross-section_area', 'Net_heat_production_from_gas-phase_reactions', 'EINO', 'EINO2', 'EINOX', 'EICO', 'Unburned_fuel_fraction', 'Temperature', 'Mole_fraction_H2', 'Mole_fraction_H', 'Mole_fraction_O', 'Mole_fraction_O2', 'Mole_fraction_OH', 'Mole_fraction_H2O', 'Mole_fraction_HO2', 'Mole_fraction_H2O2', 'Mole_fraction_C', 'Mole_fraction_CH', 'Mole_fraction_CH2', 'Mole_fraction_CH2(S)', 'Mole_fraction_CH3', 'Mole_fraction_CH4', 'Mole_fraction_CO', 'Mole_fraction_CO2', 'Mole_fraction_HCO', 'Mole_fraction_CH2O', 'Mole_fraction_CH2OH', 'Mole_fraction_CH3O', 'Mole_fraction_CH3OH', 'Mole_fraction_C2H', 'Mole_fraction_C2H2', 'Mole_fraction_C2H3', 'Mole_fraction_C2H4', 'Mole_fraction_C2H5', 'Mole_fraction_C2H6', 'Mole_fraction_HCCO', 'Mole_fraction_CH2CO', 'Mole_fraction_HCCOH', 'Mole_fraction_N', 'Mole_fraction_NH', 'Mole_fraction_NH2', 'Mole_fraction_NH3', 'Mole_fraction_NNH', 'Mole_fraction_NO', 'Mole_fraction_NO2', 'Mole_fraction

# Pre-procces
get Mass Fraction

In [29]:
# Load Molecular Weight
df_species = pd.read_csv('Species_List_GRI.csv')
num_species = df_species.shape[0]
columns_X = 'Mole_fraction_' + df_species['Name'].to_numpy()
columns_Y = 'Mass_fraction_' + df_species['Name'].to_numpy()
molecular_weights = df_species['Molecular_Weight'].to_numpy()

dfs_Y = []

for df in dfs:

    # Prepare ndarray from dataframe
    mole_fractions = df[columns_X].to_numpy()
    mean_molecular_weight = df['Molecular_weight'].to_numpy()

    # Calculate mass fraction
    mass_fractions = np.divide(np.multiply(mole_fractions, molecular_weights).T, mean_molecular_weight).T
    df_Y = pd.DataFrame(mass_fractions, columns=columns_Y)

    dfs_Y.append(df_Y)

# Test to get binary diffusion coefficient
using cantera

In [54]:
T = df['Temperature'][0]
P = df['Pressure'][0]
X = df[columns_X].to_numpy()[0, :]
Y = df_Y.to_numpy()[0, :]
dYdx = np.gradient(df_Y, df['Distance'], axis=0)[0, :]

In [14]:
gas = ct.Solution('gri30.yaml')
gas.TPX = T, P, X

In [25]:
D_ij = gas.binary_diff_coeffs

In [26]:
mu_ij = D_ij/(8.617e-5*T)

In [31]:
sum = np.zeros(num_species)

for i in range(num_species):
    for j in range(num_species):
        
        if i == j:
            continue

        sum[i] += X[i]/mu_ij[i, j]

mu_im = np.divide(1.0, sum)

In [36]:
mu_im

array([ 1.31846914e+07,  3.65483170e+18, -1.40337971e+17,  7.59735784e+00,
        4.36464409e+17,  7.92787877e+08, -1.00370982e+12,  6.23563234e+11,
        3.74421188e+18,  3.71503674e+18,  3.67483150e+18,  3.67465941e+18,
       -1.64503491e+13,  1.24581742e+01,  4.98461298e+09,  7.65587259e+10,
        3.52069758e+18,  3.64478247e+11,  3.55297059e+18,  1.51271407e+17,
        2.99668601e+12,  3.62345445e+18,  2.34984544e+12,  3.62177797e+18,
        8.49741709e+11,  3.60206770e+18,  9.30527941e+11,  3.60108607e+18,
        2.34014294e+13,  2.06029501e+16,  3.74335171e+18,  3.71055816e+18,
        6.27822621e+14,  1.01382230e+14,  3.74092544e+18, -8.63893619e+13,
        2.11872044e+13,  6.11584828e+16,  2.92230051e+16,  3.74049213e+18,
        1.23642643e+14,  2.96531572e+17,  3.60068713e+18,  6.66108794e+15,
        4.53897177e+17,  8.45562494e+14,  9.83792762e+17,  2.02985740e+00,
        3.63877181e+18,  3.85270932e+18,  3.77716182e+14,  3.53148331e+18,
        1.54065419e+14])

# Calculate Diffusion Coefficient

In [ ]:
D_dash = D - mu*Y*E/dYdx